<a href="https://cognitiveclass.ai/"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/Images/CCLog.png" width = 200, align = "center"></a>

<h1>Build a Regression Model in Keras: Project</h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">A. Build a baseline model</a>  
4. <a href="#item34">B. Normalize the data</a>  
4. <a href="#item35">C. Increate the number of epochs</a> 
4. <a href="#item36">D. Increase the number of hidden layers</a> 

</font>
</div>

<a id="item31"></a>

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [9]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

<a id="item32"></a>

## Import Keras

Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.

In [10]:
#!pip install keras
#!pip install tensorflow

#### Let's go ahead and import the Keras library

In [11]:
import keras

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

<a id="item33"></a>

## A. Build a baseline model

### Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

 - One hidden layer of 10 nodes, and a ReLU activation function
 - Use the adam optimizer and the mean squared error as the loss function.

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### Train and Test the Network

Next, we will train and test the model at the same time using the *fit* method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.

In [14]:
# fit the model
df = pd.DataFrame()
repeats = 3
for repeat in range(repeats):
    model = regression_model()  # build the model
    history_callback = model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=2)
    
    layer_name = 'repeat_{}'.format(repeat + 1) # otherwise give the layer a number
    df[layer_name] = history_callback.history["loss"]
    
    if repeat == repeats-1 :
        df['mean'] = df.mean(axis=1)
        df['std'] = df.iloc[:,0:repeats].std(axis=1)

Epoch 1/50
23/23 - 0s - loss: 34429.6797 - val_loss: 7205.2549
Epoch 2/50
23/23 - 0s - loss: 5892.3252 - val_loss: 2726.6519
Epoch 3/50
23/23 - 0s - loss: 4111.5044 - val_loss: 2064.3208
Epoch 4/50
23/23 - 0s - loss: 3240.2229 - val_loss: 1603.2306
Epoch 5/50
23/23 - 0s - loss: 2621.3755 - val_loss: 1434.1145
Epoch 6/50
23/23 - 0s - loss: 2132.5864 - val_loss: 1284.9808
Epoch 7/50
23/23 - 0s - loss: 1746.0953 - val_loss: 1169.6035
Epoch 8/50
23/23 - 0s - loss: 1450.3196 - val_loss: 1043.3615
Epoch 9/50
23/23 - 0s - loss: 1231.8783 - val_loss: 925.0154
Epoch 10/50
23/23 - 0s - loss: 1048.6973 - val_loss: 837.0942
Epoch 11/50
23/23 - 0s - loss: 897.5958 - val_loss: 747.5916
Epoch 12/50
23/23 - 0s - loss: 779.7368 - val_loss: 681.8558
Epoch 13/50
23/23 - 0s - loss: 682.2324 - val_loss: 609.2440
Epoch 14/50
23/23 - 0s - loss: 604.9186 - val_loss: 557.6312
Epoch 15/50
23/23 - 0s - loss: 539.4342 - val_loss: 507.1354
Epoch 16/50
23/23 - 0s - loss: 488.8971 - val_loss: 455.3516
Epoch 17/50
23

Epoch 35/50
23/23 - 0s - loss: 413.6663 - val_loss: 288.8494
Epoch 36/50
23/23 - 0s - loss: 394.5992 - val_loss: 279.2250
Epoch 37/50
23/23 - 0s - loss: 376.9286 - val_loss: 270.0122
Epoch 38/50
23/23 - 0s - loss: 360.7698 - val_loss: 259.3181
Epoch 39/50
23/23 - 0s - loss: 344.3469 - val_loss: 251.2223
Epoch 40/50
23/23 - 0s - loss: 330.0983 - val_loss: 242.2477
Epoch 41/50
23/23 - 0s - loss: 317.0658 - val_loss: 236.1766
Epoch 42/50
23/23 - 0s - loss: 305.3062 - val_loss: 227.2607
Epoch 43/50
23/23 - 0s - loss: 293.7560 - val_loss: 221.3172
Epoch 44/50
23/23 - 0s - loss: 282.2054 - val_loss: 214.3303
Epoch 45/50
23/23 - 0s - loss: 272.8712 - val_loss: 210.5025
Epoch 46/50
23/23 - 0s - loss: 267.5848 - val_loss: 209.8278
Epoch 47/50
23/23 - 0s - loss: 257.6196 - val_loss: 196.7141
Epoch 48/50
23/23 - 0s - loss: 246.5809 - val_loss: 194.3892
Epoch 49/50
23/23 - 0s - loss: 239.3233 - val_loss: 186.1276
Epoch 50/50
23/23 - 0s - loss: 232.6182 - val_loss: 182.0730


Report the mean and the standard deviation of the mean squared errors.

In [15]:
print(df.shape)
df

(50, 5)


,repeat_1,repeat_2,repeat_3,mean,std
0,34429.679688,87992.460938,78462.750000,66961.630208,28573.583975
1,5892.325195,23292.705078,46373.929688,25186.319987,20307.127040
2,4111.504395,7904.031738,23883.576172,11966.370768,10493.362397
3,3240.222900,6853.458984,9134.296875,6409.326253,2972.030857
4,2621.375488,6083.708008,2640.574463,3781.885986,1993.459459
5,2132.586426,5448.173828,1680.892456,3087.217570,2057.083542
6,1746.095337,4906.398926,1601.384888,2751.293050,1867.778434
7,1450.319580,4361.524414,1525.901489,2445.915161,1659.396657
8,1231.878296,3906.026123,1459.434082,2199.112834,1482.602490
9,1048.697266,3509.086670,1393.478516,1983.754150,1332.177916


<a id="item34"></a>

## B. Normalize the data

In [16]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [17]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

In [18]:
# fit the model
df_norm = pd.DataFrame()
repeats = 3
for repeat in range(repeats):
    model = regression_model()
    history_callback = model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)
    
    layer_name = 'repeat_{}'.format(repeat + 1) # otherwise give the layer a number
    df_norm[layer_name] = history_callback.history["loss"]
    
    if repeat == repeats-1 :
        df_norm['mean'] = df_norm.mean(axis=1)
        df_norm['std'] = df_norm.iloc[:,0:repeats].std(axis=1)

Epoch 1/50
23/23 - 0s - loss: 1654.9481 - val_loss: 1192.0492
Epoch 2/50
23/23 - 0s - loss: 1636.5280 - val_loss: 1178.3784
Epoch 3/50
23/23 - 0s - loss: 1617.5645 - val_loss: 1164.4292
Epoch 4/50
23/23 - 0s - loss: 1597.8940 - val_loss: 1149.9236
Epoch 5/50
23/23 - 0s - loss: 1577.2311 - val_loss: 1135.2983
Epoch 6/50
23/23 - 0s - loss: 1555.7318 - val_loss: 1119.4343
Epoch 7/50
23/23 - 0s - loss: 1533.1434 - val_loss: 1103.4462
Epoch 8/50
23/23 - 0s - loss: 1509.5677 - val_loss: 1086.7029
Epoch 9/50
23/23 - 0s - loss: 1484.5310 - val_loss: 1069.0197
Epoch 10/50
23/23 - 0s - loss: 1458.1587 - val_loss: 1051.2563
Epoch 11/50
23/23 - 0s - loss: 1430.5098 - val_loss: 1032.3098
Epoch 12/50
23/23 - 0s - loss: 1401.2866 - val_loss: 1013.4681
Epoch 13/50
23/23 - 0s - loss: 1371.4725 - val_loss: 993.5945
Epoch 14/50
23/23 - 0s - loss: 1339.7592 - val_loss: 973.2398
Epoch 15/50
23/23 - 0s - loss: 1307.5854 - val_loss: 952.3077
Epoch 16/50
23/23 - 0s - loss: 1273.6418 - val_loss: 931.2615
Epoch

Epoch 34/50
23/23 - 0s - loss: 715.5382 - val_loss: 717.8964
Epoch 35/50
23/23 - 0s - loss: 687.9860 - val_loss: 700.2939
Epoch 36/50
23/23 - 0s - loss: 661.5745 - val_loss: 683.4928
Epoch 37/50
23/23 - 0s - loss: 635.9660 - val_loss: 666.1486
Epoch 38/50
23/23 - 0s - loss: 611.2286 - val_loss: 649.1473
Epoch 39/50
23/23 - 0s - loss: 587.5609 - val_loss: 632.4630
Epoch 40/50
23/23 - 0s - loss: 564.5193 - val_loss: 616.5649
Epoch 41/50
23/23 - 0s - loss: 542.6450 - val_loss: 600.1851
Epoch 42/50
23/23 - 0s - loss: 521.8179 - val_loss: 584.4454
Epoch 43/50
23/23 - 0s - loss: 501.9438 - val_loss: 569.1311
Epoch 44/50
23/23 - 0s - loss: 482.8821 - val_loss: 553.8461
Epoch 45/50
23/23 - 0s - loss: 465.0396 - val_loss: 538.9094
Epoch 46/50
23/23 - 0s - loss: 447.8897 - val_loss: 524.6554
Epoch 47/50
23/23 - 0s - loss: 431.5663 - val_loss: 510.8921
Epoch 48/50
23/23 - 0s - loss: 416.3226 - val_loss: 497.4547
Epoch 49/50
23/23 - 0s - loss: 402.0028 - val_loss: 484.2351
Epoch 50/50
23/23 - 0s -

Report the mean and the standard deviation of the mean squared errors.

In [19]:
print(df_norm.shape)
df_norm

(50, 5)


,repeat_1,repeat_2,repeat_3,mean,std
0,1654.948120,1739.501099,1690.840820,1695.096680,42.436845
1,1636.527954,1722.459106,1667.558594,1675.515218,43.514614
2,1617.564453,1706.031738,1644.191040,1655.929077,45.386686
3,1597.894043,1690.145142,1620.978149,1636.339111,48.005579
4,1577.231079,1674.554932,1597.788452,1616.524821,51.295923
5,1555.731812,1658.732178,1574.442627,1596.302205,54.869388
6,1533.143433,1643.050415,1550.333740,1575.509196,59.120545
7,1509.567749,1626.659790,1526.305786,1554.177775,63.326711
8,1484.531006,1610.094971,1501.142456,1531.922811,68.206671
9,1458.158691,1592.471191,1475.446045,1508.691976,73.067988


#### How does the mean of the mean squared errors compare to that from Step A?

In step A the mean square error in its first iterations was much higher compared to step B, however, at the end of the 50 times, step A had a smaller error, this implies that in the end the non-normalized data give better results. results in this case.

<a id="item35"></a>

## C. Increate the number of epochs

Using 100 epochs this time for training

In [20]:
# fit the model
df_norm = pd.DataFrame()
repeats = 3
for repeat in range(repeats):
    model = regression_model()
    history_callback = model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)
    
    layer_name = 'repeat_{}'.format(repeat + 1) # otherwise give the layer a number
    df_norm[layer_name] = history_callback.history["loss"]
    
    if repeat == repeats-1 :
        df_norm['mean'] = df_norm.mean(axis=1)
        df_norm['std'] = df_norm.iloc[:,0:repeats].std(axis=1)

Epoch 1/100
23/23 - 0s - loss: 1667.7979 - val_loss: 1183.5328
Epoch 2/100
23/23 - 0s - loss: 1647.6486 - val_loss: 1168.8748
Epoch 3/100
23/23 - 0s - loss: 1627.2573 - val_loss: 1154.1864
Epoch 4/100
23/23 - 0s - loss: 1606.3318 - val_loss: 1139.0001
Epoch 5/100
23/23 - 0s - loss: 1584.5892 - val_loss: 1123.7074
Epoch 6/100
23/23 - 0s - loss: 1562.0723 - val_loss: 1108.2302
Epoch 7/100
23/23 - 0s - loss: 1538.7389 - val_loss: 1092.1567
Epoch 8/100
23/23 - 0s - loss: 1514.5055 - val_loss: 1075.5554
Epoch 9/100
23/23 - 0s - loss: 1489.2413 - val_loss: 1058.9531
Epoch 10/100
23/23 - 0s - loss: 1462.8254 - val_loss: 1041.8046
Epoch 11/100
23/23 - 0s - loss: 1435.5707 - val_loss: 1024.3715
Epoch 12/100
23/23 - 0s - loss: 1407.3275 - val_loss: 1005.8859
Epoch 13/100
23/23 - 0s - loss: 1378.0555 - val_loss: 987.4186
Epoch 14/100
23/23 - 0s - loss: 1348.4230 - val_loss: 968.5565
Epoch 15/100
23/23 - 0s - loss: 1317.6980 - val_loss: 949.2562
Epoch 16/100
23/23 - 0s - loss: 1287.2133 - val_loss

23/23 - 0s - loss: 811.8577 - val_loss: 670.0251
Epoch 33/100
23/23 - 0s - loss: 777.2335 - val_loss: 650.0766
Epoch 34/100
23/23 - 0s - loss: 742.6596 - val_loss: 630.3504
Epoch 35/100
23/23 - 0s - loss: 709.2773 - val_loss: 610.7482
Epoch 36/100
23/23 - 0s - loss: 676.3238 - val_loss: 591.3393
Epoch 37/100
23/23 - 0s - loss: 644.9983 - val_loss: 571.8842
Epoch 38/100
23/23 - 0s - loss: 613.7666 - val_loss: 552.6703
Epoch 39/100
23/23 - 0s - loss: 584.1652 - val_loss: 534.3016
Epoch 40/100
23/23 - 0s - loss: 555.8149 - val_loss: 515.9534
Epoch 41/100
23/23 - 0s - loss: 528.7519 - val_loss: 497.9706
Epoch 42/100
23/23 - 0s - loss: 503.1714 - val_loss: 480.3805
Epoch 43/100
23/23 - 0s - loss: 478.5361 - val_loss: 464.2546
Epoch 44/100
23/23 - 0s - loss: 456.0822 - val_loss: 447.0589
Epoch 45/100
23/23 - 0s - loss: 434.1750 - val_loss: 431.2162
Epoch 46/100
23/23 - 0s - loss: 414.0988 - val_loss: 415.9479
Epoch 47/100
23/23 - 0s - loss: 395.2899 - val_loss: 400.9330
Epoch 48/100
23/23 - 

Epoch 64/100
23/23 - 0s - loss: 255.2610 - val_loss: 276.6278
Epoch 65/100
23/23 - 0s - loss: 250.6940 - val_loss: 270.2590
Epoch 66/100
23/23 - 0s - loss: 246.5369 - val_loss: 264.3174
Epoch 67/100
23/23 - 0s - loss: 242.4501 - val_loss: 258.3943
Epoch 68/100
23/23 - 0s - loss: 238.7345 - val_loss: 253.6198
Epoch 69/100
23/23 - 0s - loss: 235.2424 - val_loss: 248.0445
Epoch 70/100
23/23 - 0s - loss: 231.7173 - val_loss: 243.6356
Epoch 71/100
23/23 - 0s - loss: 228.5573 - val_loss: 239.1288
Epoch 72/100
23/23 - 0s - loss: 225.4667 - val_loss: 234.9981
Epoch 73/100
23/23 - 0s - loss: 222.4892 - val_loss: 230.7156
Epoch 74/100
23/23 - 0s - loss: 219.5216 - val_loss: 226.8733
Epoch 75/100
23/23 - 0s - loss: 216.8838 - val_loss: 223.2368
Epoch 76/100
23/23 - 0s - loss: 214.3104 - val_loss: 219.0152
Epoch 77/100
23/23 - 0s - loss: 211.6296 - val_loss: 216.3257
Epoch 78/100
23/23 - 0s - loss: 209.3219 - val_loss: 212.8776
Epoch 79/100
23/23 - 0s - loss: 206.8849 - val_loss: 210.3243
Epoch 80

Report the mean and the standard deviation of the mean squared errors.

In [21]:
print(df_norm.shape)
df_norm

(100, 5)


,repeat_1,repeat_2,repeat_3,mean,std
0,1667.797852,1663.042358,1701.296631,1677.378947,20.849350
1,1647.648560,1645.997437,1683.143677,1658.929891,20.985998
2,1627.257324,1629.335083,1665.197876,1640.596761,21.330504
3,1606.331787,1612.677856,1647.015137,1622.008260,21.887806
4,1584.589233,1595.518555,1628.833862,1602.980550,23.046862
...,...,...,...,...,...
95,177.451462,172.827240,173.747025,174.675242,2.447866
96,176.003952,171.449219,172.083481,173.178884,2.467049
97,174.658524,170.055756,170.574738,171.763006,2.520983
98,173.341476,168.809921,169.048752,170.400050,2.550148


#### How does the mean of the mean squared errors compare to that from Step B?

Using 100 epochs, better results are obtained, achieving better values than in step A and step B, although the values are close to step A, in step C a much smaller standard deviation is obtained.

<a id="item36"></a>

## D. Increase the number of hidden layers 

In [22]:
# define regression model
def regression_model_B():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [23]:
# fit the model
df_norm = pd.DataFrame()
repeats = 3
for repeat in range(repeats):
    model = regression_model_B()
    history_callback = model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)
    
    layer_name = 'repeat_{}'.format(repeat + 1) # otherwise give the layer a number
    df_norm[layer_name] = history_callback.history["loss"]
    
    if repeat == repeats-1 :
        df_norm['mean'] = df_norm.mean(axis=1)
        df_norm['std'] = df_norm.iloc[:,0:repeats].std(axis=1)

Epoch 1/50
23/23 - 0s - loss: 1707.1511 - val_loss: 1226.1625
Epoch 2/50
23/23 - 0s - loss: 1684.9954 - val_loss: 1206.8888
Epoch 3/50
23/23 - 0s - loss: 1657.2130 - val_loss: 1181.4323
Epoch 4/50
23/23 - 0s - loss: 1617.2899 - val_loss: 1145.4061
Epoch 5/50
23/23 - 0s - loss: 1561.1637 - val_loss: 1096.4148
Epoch 6/50
23/23 - 0s - loss: 1484.7805 - val_loss: 1031.6111
Epoch 7/50
23/23 - 0s - loss: 1381.6160 - val_loss: 946.0930
Epoch 8/50
23/23 - 0s - loss: 1243.9788 - val_loss: 838.0406
Epoch 9/50
23/23 - 0s - loss: 1065.6915 - val_loss: 703.3225
Epoch 10/50
23/23 - 0s - loss: 849.5839 - val_loss: 546.9659
Epoch 11/50
23/23 - 0s - loss: 624.0748 - val_loss: 398.6194
Epoch 12/50
23/23 - 0s - loss: 430.2958 - val_loss: 290.9235
Epoch 13/50
23/23 - 0s - loss: 308.4342 - val_loss: 232.2239
Epoch 14/50
23/23 - 0s - loss: 249.0766 - val_loss: 211.6922
Epoch 15/50
23/23 - 0s - loss: 224.3861 - val_loss: 200.8786
Epoch 16/50
23/23 - 0s - loss: 211.7305 - val_loss: 195.9780
Epoch 17/50
23/23 

23/23 - 0s - loss: 156.6962 - val_loss: 149.1887
Epoch 36/50
23/23 - 0s - loss: 154.5953 - val_loss: 147.8427
Epoch 37/50
23/23 - 0s - loss: 152.2889 - val_loss: 145.3190
Epoch 38/50
23/23 - 0s - loss: 149.7756 - val_loss: 143.2502
Epoch 39/50
23/23 - 0s - loss: 147.7473 - val_loss: 141.7266
Epoch 40/50
23/23 - 0s - loss: 145.2357 - val_loss: 141.6100
Epoch 41/50
23/23 - 0s - loss: 142.9976 - val_loss: 139.7146
Epoch 42/50
23/23 - 0s - loss: 140.7312 - val_loss: 138.2389
Epoch 43/50
23/23 - 0s - loss: 138.7255 - val_loss: 137.5203
Epoch 44/50
23/23 - 0s - loss: 136.3338 - val_loss: 134.4845
Epoch 45/50
23/23 - 0s - loss: 134.4210 - val_loss: 133.0623
Epoch 46/50
23/23 - 0s - loss: 132.4233 - val_loss: 132.8895
Epoch 47/50
23/23 - 0s - loss: 129.8069 - val_loss: 129.0739
Epoch 48/50
23/23 - 0s - loss: 127.6919 - val_loss: 129.7571
Epoch 49/50
23/23 - 0s - loss: 125.1745 - val_loss: 127.7702
Epoch 50/50
23/23 - 0s - loss: 123.0651 - val_loss: 126.2103


Report the mean and the standard deviation of the mean squared errors.

In [24]:
print(df_norm.shape)
df_norm

(50, 5)


,repeat_1,repeat_2,repeat_3,mean,std
0,1707.151123,1688.064453,1697.474121,1697.563232,9.543647
1,1684.995361,1663.554932,1677.417603,1675.322632,10.872658
2,1657.213013,1624.632568,1654.494629,1645.446737,18.076770
3,1617.289917,1562.842285,1625.251953,1601.794718,33.967890
4,1561.163696,1463.162598,1585.213257,1536.513184,64.651579
5,1484.780518,1316.192383,1528.594604,1443.189168,112.143017
6,1381.615967,1116.552368,1448.077637,1315.415324,175.397106
7,1243.978760,878.562500,1327.543457,1150.028239,238.780234
8,1065.691528,641.884827,1151.186157,952.920837,272.735922
9,849.583923,460.582703,921.082336,743.749654,247.821806


#### How does the mean of the mean squared errors compare to that from Step B?

The D step obtained the best results in comparison to the other steps, however a variation in the standard deviation is observed with behavior in its increasing and decreasing epochs.